# Explication de modèles NLP

Tous les modèles d'apprentissage automatique qui opèrent dans des dimensions supérieures à ce qui peut être directement visualisé par l'esprit humain peuvent être qualifiés de modèles boîte noire, ce qui affecte l'interprétabilité des modèles. En particulier dans le domaine du traitement du langage naturel (NLP), il est toujours le cas que les dimensions des caractéristiques sont très grandes, ce qui rend beaucoup plus compliquée l'explication de l'importance des caractéristiques.

Ainsi ce notebook concistera à construire un modèle de classification de texte multi-classes, puis appliquer séparément LIME et SHAP pour l'explication et l'interprétabilité des modèles.

### 1- Importation des bibliothèques

In [1]:
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf

from sklearn.model_selection import train_test_split
from ast import literal_eval

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import tensorflow as tf

### 2- Chargement et préparation des données

In [ ]:
df=pd.read_csv('EFREI - LIPSTIP - 50k elements EPO.csv')
df.info()

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df=df.drop(["Numero de publication","date de publication","IPC","Numéro d'application", "Date d'application"],axis=1)

In [ ]:
df.isnull().sum()

La base de données ne contient aucune informations manquante

In [ ]:
total_duplicate = sum(df.duplicated())
print(f"There are {total_duplicate} duplicate data.")

Les données sont mélangées pour s'assurer qu'elles sont réparties aléatoirement.

In [ ]:
from sklearn.utils import shuffle
df = shuffle(df,random_state=101)

On utilise les 20 000 premières lignes pour l'entraînement.

In [ ]:
df = df[:20000]
print(f"There are {len(df)} rows in the dataset.")

Création d'une nouvelle colonne pour les sous-classes CPC et conversion en liste

### 3- Préparation des labels et nettoyage des données

 Les codes CPC sont extraits et transformés en liste, puis tronqués à quatre caractères, et stockés dans une nouvelle colonne CPC_tronc.

In [ ]:
from ast import literal_eval
labels = df["CPC"].str.split(",")

In [ ]:
labels = df["CPC"].apply(literal_eval)
print(labels.values[:1])

In [ ]:
labels

In [ ]:
def tronquer_codes_cpc(codes):
    return [[code[:4] for code in liste_codes ]  for liste_codes in codes]
            
moi=tronquer_codes_cpc(labels)
moi

In [ ]:
df["CPC_tronc"]=moi

In [ ]:
print(df["CPC_tronc"])

### Nettoyage du texte

Il définit une fonction clean_text(text) qui effectue plusieurs opérations de nettoyage sur chaque texte dans la colonne 'post' du dataframe :
    
Décoder le texte HTML en texte brut en utilisant BeautifulSoup.

Convertir le texte en minuscules.

Remplacer certains symboles spécifiques par des espaces.

Supprimer certains symboles considérés comme indésirables.

Supprimer les stopwords

In [ ]:
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
import re

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    text = BeautifulSoup(text, "lxml").text 
    text = text.lower() 
    text = REPLACE_BY_SPACE_RE.sub(' ', text) 
    text = BAD_SYMBOLS_RE.sub('', text) 
    text = ' '.join(word for word in text.split() if word not in STOPWORDS)
    return text


In [ ]:
df['description'] = df['description'].apply(clean_text)
#print(df['description'].iloc[0])

### 4- Division du jeu de données

Cette partie conciste à Nous allons maintenant diviser le jeu de données en ensembles d'entraînement, de validation et de test en utilisant 10 % des données pour le test et 50 % des données de test pour l'ensemble de validation.

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=10000) 
tokenizer.fit_on_texts(df['description'])

sequences = tokenizer.texts_to_sequences(df['description'])
sequences = pad_sequences(sequences, maxlen=200)  

x_train = sequences[:int(0.8 * len(sequences))]
x_val = sequences[int(0.8 * len(sequences)):]

y_train = to_categorical(df['CPC_tronc'].values[:int(0.8 * len(df))], num_classes=len(set(df['CPC_tronc'].values)))
y_val = to_categorical(df['CPC_tronc'].values[int(0.8 * len(df)):], num_classes=len(set(df['CPC_tronc'].values)))


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=64),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(len(set(df['CPC_tronc'].values)), activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
history = model.fit(x_train, y_train, epochs=10, validation_data=(x_val, y_val))

In [ ]:
x_test = sequences[int(0.8 * len(sequences)):]
y_test = to_categorical(df['CPC_tronc'].values[int(0.8 * len(df)) :], num_classes=len(set(df['CPC_tronc'].values)))

loss, accuracy = model.evaluate(x_test, y_test)
print('Perte:', loss)
print('Précision:', accuracy)


### 5- Préparation des labels pour le modèle

Dans cette étape, nous allons préparer les étiquettes pour l'entraînement en utilisant la couche StringLookup de TensorFlow, qui convertit les labels sous forme de texte en encodages numériques (multi-hot) que le modèle peut utiliser.

Multi-hot : Chaque label est représenté par un vecteur binaire où chaque dimension correspond à un label possible. Si un document est associé à plusieurs labels, ces labels sont tous activés dans le vecteur multi-hot.
Cela permet au modèle de traiter des problèmes de classification multi-label, où un échantillon peut appartenir à plusieurs catégories simultanément.


On extrait le vocabulaire (les labels uniques) de la couche StringLookup. Ensuite, on nettoie les entrées du vocabulaire en supprimant les espaces et les caractères superflus pour obtenir une liste propre de labels uniques.

La fonction invert_multi_hot permet de convertir un vecteur multi-hot encodé en une liste de labels lisibles, facilitant ainsi l'interprétation des résultats du modèle.

In [ ]:
Subclass_labels = tf.ragged.constant(train_df["CPC_tronc"].values)
lookup = tf.keras.layers.StringLookup(output_mode="multi_hot")
lookup.adapt(Subclass_labels)
vocab = lookup.get_vocabulary()
cleaned_vocab = [entry.strip().strip("[]'") for entry in vocab if entry.strip()]


def invert_multi_hot(encoded_labels):
    """Reverse a single multi-hot encoded label to a tuple of vocab Subclass_labels."""
    hot_indices = np.argwhere(encoded_labels == 1.0)[..., 0]
    return np.take(cleaned_vocab, hot_indices)


print("Vocabulary:\n")
#print(cleaned_vocab)

In [ ]:
print(len(cleaned_vocab)) 

Un exemple pour montrer comment une étiquette de sous-classe est extraite d'un DataFrame, puis transformée en une représentation binarisée utilisant la couche lookup. 

In [ ]:
sample_label = train_df["CPC_tronc"].iloc[0]
print(f"Original label: {sample_label}")

label_binarized = lookup([sample_label])
print(f"Label-binarized representation: {label_binarized}")

### 6- Préparation des jeux de données pour TensorFlow

Nous devons préparer les textes en assurant qu'ils ont une longueur unifiée pour pouvoir les passer dans le modèle. Pour cela, nous allons:Définir la longueur maximale de séquence et le batch size, puis la fonction unify_text_length sépare le texte en mots, calcule sa longueur et la quantité de padding nécessaire

Notre max_seqlen fixé à 400, alors tous les textes plus longs seront tronqués pour ne garder que les 400 premiers mots, et les textes plus courts seront remplis avec des tokens spéciaux (comme <pad>) pour atteindre cette longueur maximale.

Uniformiser la longueur des séquences de texte pour qu'elles aient toutes la même longueur, facilitant ainsi leur traitement par le modèle de machine learning. En effet, pour qu'un batch de données puisse être traité efficacement par un modèle, chaque exemple dans le batch doit avoir la même forme.

In [ ]:
max_seqlen = 400
batch_size = 32
padding_token = "<pad>"
auto = tf.data.AUTOTUNE

def unify_text_length(text, label):
    word_splits = tf.strings.split(text, sep=" ")
    sequence_length = tf.shape(word_splits)[0]

    padding_amount = max_seqlen - sequence_length

    if padding_amount > 0:
        unified_text = tf.pad([text], [[0, padding_amount]], constant_values=padding_token)
        unified_text = tf.strings.reduce_join(unified_text, separator="")
    else:
        unified_text = tf.strings.reduce_join(word_splits[:max_seqlen], separator=" ")

    return tf.expand_dims(unified_text, -1), label


La fonction make_dataset crée un dataset TensorFlow prêt pour l'entraînement, la validation et le test en transformant les textes et les labels en un format que le modèle peut traiter efficacement.

In [ ]:
def make_dataset(dataframe, is_train=True):
    labels = tf.ragged.constant(dataframe["CPC_tronc"].values)
    label_binarized = lookup(labels).numpy()
    dataset = tf.data.Dataset.from_tensor_slices(
        (dataframe["description"].values, label_binarized)
    )
    dataset = dataset.shuffle(batch_size * 10) if is_train else dataset
    dataset = dataset.map(unify_text_length, num_parallel_calls=auto).cache()
    return dataset.batch(batch_size)

In [ ]:
train_dataset = make_dataset(train_df, is_train=True)
validation_dataset = make_dataset(val_df, is_train=False)
test_dataset = make_dataset(test_df, is_train=False)

In [ ]:
train_dataset

Chaque élément de train_dataset est un tuple contenant deux éléments : un tenseur représentant un texte (tf.Tensor de type string).
et un tenseur représentant les labels encodés en multi-hot (tf.Tensor de type int64).

#### Quelques exemples de textes et leurs labels correspondants après le traitement

In [ ]:
text_batch, label_batch = next(iter(train_dataset))

for i, text in enumerate(text_batch[:3]):
    label = label_batch[i].numpy()[None, ...]
    print(f"description: {text[0]}")
    print(f"Label(s): {invert_multi_hot(label[0])}")
    print(" ")

### 7- Vectorisation des textes

Cette partie du code calcule la taille maximale du vocabulaire basé sur la longueur des textes dans le dataframe d'entraînement (train_df) et ressort la taille du plus long texte

Notre vocabulary_size est 10571, cela signifie que le modèle ne prendra en compte que les 10571 mots les plus fréquents (ou n-grammes) dans l'ensemble de données. Les mots moins fréquents peuvent être ignorés

In [ ]:
train_df["total_words"] = train_df["description"].str.split().str.len()
vocabulary_size = train_df["total_words"].max()
print(f"Vocabulary size: {vocabulary_size}")

Une couche TextVectorization de Keras est utilisée pour convertir les textes en vecteurs de TF-IDF. map : Applique une fonction à chaque élément du dataset. Ici, la fonction lambda applique text_vectorizer pour transformer les textes en vecteurs de TF-IDF.


 Utiliser des n-grammes et TF-IDF permet de capturer des informations importantes sur la fréquence des mots et leur importance relative dans le corpus, ce qui peut améliorer la performance du modèle.

In [ ]:
text_vectorizer = layers.TextVectorization(
    max_tokens=vocabulary_size, ngrams=2, output_mode="tf_idf"
)

with tf.device("/CPU:0"):
    text_vectorizer.adapt(train_dataset.map(lambda text, label: text))

train_dataset = train_dataset.map(
    lambda text, label: (text_vectorizer(text), label), num_parallel_calls=auto
).prefetch(auto)
validation_dataset = validation_dataset.map(
    lambda text, label: (text_vectorizer(text), label), num_parallel_calls=auto
).prefetch(auto)
test_dataset = test_dataset.map(
    lambda text, label: (text_vectorizer(text), label), num_parallel_calls=auto
).prefetch(auto)

### 8- Définition et compilation du modèle

Le modèle de deep learning utilisé dans votre programme est un réseau de neurones multicouche (MLP pour Multi-Layer Perceptron). Lorsque des données sont introduites dans le modèle, elles passent à travers chaque couche.
Chaque couche applique une transformation linéaire suivie d'une fonction d'activation non linéaire (dans ce cas, "relu" pour les couches cachées et "sigmoid" pour la couche de sortie).

make_model() : Cette fonction crée et retourne un modèle séquentiel (Sequential) de Keras, qui est une séquence linéaire de couches.Elles ajoute 3 couches denses.  

Première couche dense : Composée de 512 neurones avec une fonction d'activation ReLU. Cela permet au modèle d'apprendre des représentations non linéaires à partir des données.

Deuxième couche dense : Composée de 256 neurones avec une fonction d'activation ReLU. Cette couche réduit progressivement la dimensionnalité des représentations apprises par le modèle.

Troisième couche dense : C'est la couche de sortie du modèle. Le nombre de neurones correspond à la taille du vocabulaire déterminé par lookup.vocabulary_size(), et l'activation "sigmoid" est utilisée pour une classification multi-label où chaque neurone de sortie prédit une probabilité indépendante d'appartenance à une classe.

In [ ]:
def make_model():
    shallow_mlp_model = keras.Sequential(
        [
            layers.Dense(512, activation="relu"),
            layers.Dense(256, activation="relu"),
            layers.Dense(lookup.vocabulary_size(), activation="sigmoid"),
        ]  
    )
    return shallow_mlp_model

recall_m et precision_m: Calculent respectvement le rappel et la précision en utilisant la formule standard basée sur les vrais positifs et les faux négatifs.

f1_m : Calcule la F1-score en utilisant la formule harmonique de la précision et du rappel

In [ ]:
import tensorflow.keras.backend as K

def recall_m(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

 La fonction make_model() retourne un modèle séquentiel (Sequential) de Keras, qui est une séquence linéaire de couches. Ce modèle est assigné à la variable shallow_mlp_model

loss="binary_crossentropy" : Une fois que les données ont été propagées à travers le modèle et que les prédictions sont générées, la fonction de perte "binary_crossentropy" mesure à quel point les prédictions du modèle sont éloignées des étiquettes réelles.

optimizer="adam" : C'est l'optimiseur utilisé pour minimiser la fonction de perte. "adam" est un optimiseur populaire en apprentissage profond en raison de son efficacité et de sa capacité à gérer les grands ensembles de données et les réseaux de neurones profonds.

In [ ]:
shallow_mlp_model = make_model()
shallow_mlp_model.compile(
    loss="binary_crossentropy", optimizer="adam", metrics=["categorical_accuracy",f1_m,precision_m, recall_m]
)

### 9- Entraînement du modèle

epochs = 2 signifie que le modèle sera entraîné sur l'ensemble des données d'entraînement deux fois. Plus le nombre d'époques est élevé, plus le modèle a d'opportunités d'apprendre à partir des données, mais il doit être choisi judicieusement pour éviter le surapprentissage.

In [ ]:
epochs=2
history = shallow_mlp_model.fit(
    train_dataset, validation_data=validation_dataset, epochs=epochs
)

Le nombre 563 dans indique le nombre total de batches que le modèle a traités pendant une époque complète d'entraînement, sachant qu'on a un dataset d'entrainement de 9000 textes et que chaque batch contient 32 échantillons ( batc_size = 32) 

### 10-Visualisation des performances du modèle

In [ ]:
from tensorflow.keras.utils import plot_model

In [ ]:
a = len(history.history["loss"])

In [ ]:
plt.figure(figsize=(15,6))
plt.subplot(1, 2, 1) 
plt.plot(history.history["loss"], label="train_loss")
plt.plot(history.history["val_loss"], label="val_loss")
plt.xlabel("Epochs", fontsize=16)
plt.ylabel("loss", fontsize=16)
plt.xticks(np.arange(a), np.arange(1, a+1), fontsize=12)
plt.yticks(fontsize=12)
plt.title("Train and Validation loss Over Epochs", fontsize=14)
plt.legend()
plt.grid()

# plt.show()
plt.subplot(1, 2, 2) 
plt.plot(history.history["f1_m"], label="f1_m")
plt.plot(history.history["val_f1_m"], label="val_f1_m")
plt.xlabel("Epochs",fontsize=16)
plt.ylabel("accuracy",fontsize=16)
plt.xticks(np.arange(a), np.arange(1, a+1), fontsize=12)
plt.yticks(fontsize=12)
plt.title("Train and Validation f1 score Over Epochs", fontsize=14)
plt.legend()
plt.grid()

#### Évaluation du modèle

In [ ]:
_, categorical_acc, f1_m, precision_m, recall_m = shallow_mlp_model.evaluate(test_dataset)
print(f"accuracy on the test set: {round(categorical_acc * 100, 2)}%.")
print(f"f1 on the test set: {round(f1_m * 100, 2)}%.")
print(f"precision on the test set: {round(precision_m * 100, 2)}%.")
print(f"recall on the test set: {round(recall_m * 100, 2)}%.")

#### Inférence avec le modèle

Un petit ensemble de données est utilisé pour montrer comment le modèle fait des prédictions. Pour chaque texte, les labels réels et les labels prédits sont affichés.

In [ ]:
model_for_inference = keras.Sequential([text_vectorizer, shallow_mlp_model])

inference_dataset = make_dataset(test_df.sample(100), is_train=False)
text_batch, label_batch = next(iter(inference_dataset))
predicted_probabilities = model_for_inference.predict(text_batch)

for i, text in enumerate(text_batch[:5]):
    label = label_batch[i].numpy()[None, ...]
    print(f"Abstract: {text[0]}")
    print(f"Label(s): {invert_multi_hot(label[0])}")
    predicted_proba = [proba for proba in predicted_probabilities[i]]
    top_3_labels = [
        x
        for _, x in sorted(
            zip(predicted_probabilities[i], lookup.get_vocabulary()),
            key=lambda pair: pair[0],
            reverse=True,
        )
    ][:3]
    print(f"Predicted Label(s): ({', '.join([label for label in top_3_labels])})")
    print(" ")